# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [130]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.grid_search import GridSearchCV


from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit

In [131]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [132]:
student_data[student_data['passed'] == 'yes'].shape[0]

265

In [211]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = n_passed / float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate*100)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [134]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [135]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [136]:
def reformat(col_data):
    return col_data.replace(['yes', 'no'], [1, 0])

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [137]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
#student_data['passed'] = format(student_data['passed'])
y = student_data['passed']

def stratify_and_shuffle(X,y,num_train):
    sss = StratifiedShuffleSplit(y, 1, train_size=num_train)

    # only one iteration
    for train_index, test_index in sss:
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = stratify_and_shuffle(X_all, y, num_train)
    
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 40 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [138]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    diff = end - start
    print "Done!\nTraining time (secs): {:.3f}".format(diff)
    return diff

# TODO: Choose a model, import it and instantiate an object

from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=30)

# Fit model to training data
train_classifier(rf_clf, X_train, y_train)  # note: using entire training set here
print rf_clf  # you can inspect the learned model by printing it

Training RandomForestClassifier...
Done!
Training time (secs): 0.018
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [139]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    prediction_time_diff = end - start
    print "Done!\nPrediction time (secs): {:.3f}".format(prediction_time_diff)
    return (f1_score(target.values, y_pred, pos_label='yes'), prediction_time_diff)

train_f1_score = predict_labels(rf_clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score[0])

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 1.0


In [140]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(rf_clf, X_test, y_test))

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: (0.73333333333333328, 0.0017669200897216797)


In [141]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_diff = train_classifier(clf, X_train, y_train)
    
    f1_score_train, pred_time_train = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {}".format(f1_score_train)
    
    f1_score_test, pred_time_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(f1_score_test)
    
    return (f1_score_train, f1_score_test, train_diff, pred_time_test)

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [142]:
# TODO: Train and predict using two other models
from sklearn.ensemble import AdaBoostClassifier
ab_clf = AdaBoostClassifier(n_estimators=3)
print ab_clf
train_predict(ab_clf, X_train, y_train, X_test, y_test)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=3, random_state=None)
------------------------------------------
Training set size: 300
Training AdaBoostClassifier...
Done!
Training time (secs): 0.003
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.824833702882
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.813559322034


(0.82483370288248337,
 0.81355932203389825,
 0.002950906753540039,
 0.0004858970642089844)

In [143]:
# purpose is mainly to average out run times.
# Run times might defer on GC and other system-related processes.

def run_all(classifiers, sizes):
    for clf in classifiers:
        for size in sizes:
            y = student_data['passed']
            X_train, X_test, y_train, y_test = stratify_and_shuffle(X_all, y, size)
           
            total_training_time = 0.0
            total_prediction_time = 0.0
            total_f1_train_score = 0.0
            total_f1_test_score = 0.0
        
            num_times_to_run = 10
            for x in range(0, num_times_to_run): 
                
            
                f1_score_train, f1_score_test, train_time, pred_time_test = train_predict(clf, X_train, y_train, X_test, y_test)
                total_f1_train_score += f1_score_train
                total_f1_test_score += f1_score_test
                total_prediction_time += pred_time_test
                total_training_time += train_time

            avg_f1_train_score = total_f1_train_score / float(num_times_to_run)
            avg_f1_test_score = total_f1_test_score / float(num_times_to_run)
            avg_prediction_time = total_prediction_time / float(num_times_to_run)
            avg_training_time= total_training_time / float(num_times_to_run)
            
            print "\n"
            print "This run has an average training time of:     {}".format(avg_training_time)
            print "This run has an average prediction time of:   {}".format(avg_prediction_time)
            print "This run has an average f1 training score of: {}".format(avg_f1_train_score)
            print "This run has an average f1 test score of:     {}".format(avg_f1_test_score)
            print "\n\n"
                        

In [144]:
from sklearn.naive_bayes import MultinomialNB
mb_clf = MultinomialNB(alpha=10)
print mb_clf
train_predict(mb_clf, X_train, y_train, X_test, y_test)

MultinomialNB(alpha=10, class_prior=None, fit_prior=True)
------------------------------------------
Training set size: 300
Training MultinomialNB...
Done!
Training time (secs): 0.001
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.809954751131
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.833333333333


(0.80995475113122173,
 0.83333333333333337,
 0.001486063003540039,
 0.00012302398681640625)

In [145]:
run_all([rf_clf, ab_clf, mb_clf], [100, 200, 300])

------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.017
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.71186440678
------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.014
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.741935483871
------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.014
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 1

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [146]:
# TODO: Fine-tune your model and report the best F1 score
def run_grid_search(num_times):
    f1_scores = []
    alphas = []
    
    for num in range(0,num_times):
        parameters = {'alpha':[0.01, 0.05 ,0.1, 1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] }
        y = reformat(student_data['passed'])

        clf = GridSearchCV(mb_clf, parameters, scoring='f1')
        X_train, X_test, y_train, y_test = stratify_and_shuffle(X_all, y, 300)
        clf.fit(X_train, y_train)
        f1_scores.append(clf.score(X_test, y_test))
        alphas.append(clf.best_params_['alpha'])
        print clf.best_estimator_
    
    
    print "F1 Scores", pd.Series(f1_scores).describe()
    print "Alphas", pd.Series(alphas).describe()



In [147]:
run_grid_search(100)

MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=60, class_prior=None, fit_prior=True)
MultinomialNB(alpha=45, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(alpha=35, class_prior=None, fit_prior=True)
MultinomialNB(alpha=60, class_prior=None, fit_prior=True)
MultinomialNB(alpha=10, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(alpha=40, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(alpha=35, class_prior=None, fit_prior=True)
MultinomialNB(alpha=60, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(alpha=60, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(alpha=50, class_prior=None, fit_prior=True)
MultinomialNB(alpha=15, class_prior=None, fit_prior=True)
MultinomialNB(